In [1]:
class LexicalError(Exception):
    pass
class SyntaxError(Exception):
    pass

In [14]:
classes = {'L': 'abcdfghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ',
           'e':'e',
           'N': '1234567890',
           '+': '+-',
           '=': '=',
           '>': '>',
           '<': '<',
           '!': '!',
           'SS': '{}[]()/*,\n',
           '.':'.'}


def minus(class_name, lexem_table):
    if (class_name == 'N'):
        if (len(lexem_table) > 0 and lexem_table[-1][2] in [100, 101, 28]):
            return (False, True)
        else:
            return (3, True)
    elif (class_name == '.'):
        if (len(lexem_table) > 0 and lexem_table[-1][2] in [100, 101, 28]):
            return (False, True)
        else:
            return (8, True)
        
    elif(class_name == 'separator'):
        return (False, True)
        
    else:
        return (False, False)
    
    

automa = {
     1: ({'L': 2, 'N': 3, '+': 'minus_condition', '.': 8, '=': 15, '>': 12, '<': 9, '!': 18, 'SS': 20, 'e': 2}, False),
     2: ({'L': 2, 'N': 2, 'e': 2}, True, 'IDN'),
     3: ({'N': 3, 'e': 5, '.': 4}, True, 'CON'),
     4: ({'N': 4, 'e': 5}, True, 'CON'),
     5: ({'N': 6, '+': 7}, True, 'CON'),
     6: ({}, True, 'CON'),
     7: ({'N': 6}, False),
     8: ({'N': 4}, True, 'CON'),
     9: ({'<': 10, '=': 11}, True),
     10: ({}, True),
     11: ({}, True),
     12: ({'=': 14, '>': 13}, True),
     13: ({}, True),
     14: ({}, True),
     15: ({'=': 16}, True),
     16: ({}, True),
     17: ({}, True), 
     18: ({'=': 19}, False ), 
     19: ({}, True),
     20: ({}, True), 
     'minus_condition': minus
}


lexems_table = {'int': 1,
          'float': 2,
          'for': 3,
          'rof': 4,
          'while': 5,
          'do': 6,
          'print': 7,
          'input': 8,
          'by': 9,
          'if': 10,
          '\n': 12,
          ',': 13,
          '=': 14,
          '+': 15,
          '-': 16,
          '*': 17,
          '/': 18,
          '>': 19,
          '<': 20,
          '==': 21,
          '>=': 22,
          '<=': 23,
          '!=': 24,
          '[': 25,
          ']': 26,
          '(': 27,
          ')': 28,
          '{': 29,
          '}': 30,
          'to':31}

In [15]:
with open('code.txt') as file:
    code = file.read()

In [16]:
class LexicalAnalyser():
    def __init__(self, automa, classes, lexems_table, IDN, CON):
        self.automa = automa
        self.classes = classes
        self.lexems_table = lexems_table
        self.IDN_code = IDN
        self.CON_code = CON
        
        
        
    def analyse(self, code):
        state = 1
        table_of_lexems = []
        table_of_constants = []
        table_of_id = {}
        lexem = ''
        row = 1
        const_id = 0
        for char in code:
            class_name = self.get_class(char)
            if (class_name == 'separator' and state == 1):
                continue
            if(char == '\n'):
                row=row+1
    
            if(callable(self.automa[state])):
                state, is_finite = self.automa[state](class_name, table_of_lexems)
            else:
                current_state_info = self.automa[state]
                curves = self.automa[state][0]
                is_finite = self.automa[state][1]

                state = curves.get(class_name, False)
            
            if(state == False and is_finite == True):
                lexem_code = self.lexems_table.get(lexem, False)
                if (lexem_code == False):
                    if (current_state_info[2] == 'IDN'):
                        
                        if(table_of_lexems[-1][1] in ['int', 'float']):
                                if(lexem in table_of_id):
                                    raise SyntaxError('Variable {} has already been declared'.format(char))
                                   
                                lexem_code = self.IDN_code                                
                                table_of_id[lexem] = table_of_lexems[-1][1]
                                table_of_lexems.append([row, lexem, self.IDN_code, True, False])
                                
                        elif(lexem in table_of_id):
                            table_of_lexems.append([row, lexem, self.IDN_code, True, False])
                            
                        else:                                
                            print(table_of_lexems[-1][1])
                            raise SyntaxError('Undeclared variable {} is used at {} row'.format(char, row))
                        
                    else:
                        lexem_code = self.CON_code
                        table_of_constants.append([const_id, lexem, self.get_CON_type(lexem)])
                        table_of_lexems.append([row, lexem, self.CON_code, False, const_id])
                        const_id += 1
                else:
                    table_of_lexems.append([row, lexem, lexem_code, False, False])  #выделили лексему
                
                state = 1
                
                if(class_name == 'separator'):
                    lexem = ''
                    continue 
                    
                if(callable(self.automa[state])):
                    state, is_finite = self.automa[state](class_name, table_of_lexems)
                else: 
                    curves = self.automa[state][0]
                    state = curves.get(class_name, False)
                    
                if(state == False):
                    raise LexicalError("Lexical error {} at row  {}".format(char, row))
                lexem = char
            elif(state == False and is_finite == False):
                raise LexicalError("Lexical error {} at row  {}".format(char, row))
            else:
                lexem += char
        return table_of_lexems, table_of_constants, table_of_id
        
        
    def get_class(self, char):
        if (char in ' \t'):
            return 'separator'
        for key, value in self.classes.items():
            if (char in value):  
                return key
        raise LexicalError("Character '{}' does not belong to any of the classes at {} row".format(char, row) )
        
    def get_CON_type(self, lexem):
        try:
            int(lexem)
            return 'int'
        except ValueError:
            return 'float' 


In [17]:
LexicalAnalyser(automa, classes, lexems_table, 100, 101 ).analyse(code)

([[2, '\n', 12, False, False],
  [2, 'int', 1, False, False],
  [2, 'a', 100, True, False],
  [2, '=', 14, False, False],
  [3, '5', 101, False, 0],
  [3, '\n', 12, False, False],
  [3, 'float', 2, False, False],
  [3, 'b', 100, True, False],
  [3, '=', 14, False, False],
  [3, '-2.4', 101, False, 1],
  [3, '-', 16, False, False],
  [4, '3e-4', 101, False, 2],
  [4, '\n', 12, False, False],
  [4, 'int', 1, False, False],
  [4, 'i', 100, True, False],
  [4, '=', 14, False, False],
  [5, '-3', 101, False, 3],
  [6, '\n', 12, False, False],
  [7, '\n', 12, False, False],
  [7, '\n', 12, False, False],
  [7, 'for', 3, False, False],
  [7, 'i', 100, True, False],
  [7, '=', 14, False, False],
  [7, '1', 101, False, 4],
  [7, 'to', 31, False, False],
  [7, '10', 101, False, 5],
  [7, 'by', 9, False, False],
  [7, '2', 101, False, 6],
  [7, 'while', 5, False, False],
  [7, '(', 27, False, False],
  [7, 'a', 100, True, False],
  [7, '>', 19, False, False],
  [7, '2', 101, False, 7],
  [7, ')',

In [71]:
callable(minus)

True